In [ ]:
"""
PyTeapot module for drawing rotating cube using OpenGL as per
quaternion or yaw, pitch, roll angles received over serial port.
"""

from pandas.core.arrays.period import dt64arr_to_periodarr
from pandas.tseries import frequencies
import pygame
import math
from OpenGL.GL import *
from OpenGL.GLU import *
from pygame.locals import *

import socket
import numpy as np
import pandas as pd

# Define the UDP server's IP address and port to bind to
server_ip = "0.0.0.0"  # Listen on all available network interfaces
server_port = 1234  # Replace with the port you want to use

# Create a UDP socket
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# Bind the socket to the server IP and port
udp_socket.bind((server_ip, server_port))
udp_socket.settimeout(0.5)

print(f"UDP server is listening on {server_ip}:{server_port}")
Q = np.array([1., 0., 0., 0.])
ind =  (('gyro', 'x'),
        ('gyro', 'y'),
        ('gyro', 'z'),
        ('accel', 'x'),
        ('accel', 'y'),
        ('accel', 'z'),
        ('mag', 'x'),
        ('mag', 'y'),
        ('mag', 'z'))
calib = pd.Series(np.zeros(9), index=ind)

def read_data():
    data, client_address = udp_socket.recvfrom(3000)  # Adjust the buffer size as needed
    gyro = []
    accel = []
    mag = []
    data = data.decode('utf-8').splitlines()
    # print(len(data))
    try:
        for i in data:
            i = i.split(',')
            accel.append([float(i[0]), float(i[1]), float(i[2])])
            gyro.append([float(i[3]), float(i[4]), float(i[5])])
            mag.append([float(i[6]), float(i[7]), float(i[8])])
    except Exception as e:
        print("Error in reading data")
        print(i)
        print(data)
        print(len(data))
        raise e
    gyro = pd.DataFrame(gyro, columns=['x', 'y', 'z'])
    accel = pd.DataFrame(accel, columns=['x', 'y', 'z'])
    mag = pd.DataFrame(mag, columns=['x', 'y', 'z'])
    return pd.concat([gyro, accel, mag], keys=['gyro', 'accel', 'mag'], axis=1)

def calibrate(n_samples=200):
    df = pd.DataFrame()
    for i in range(n_samples):
        df = pd.concat([df, read_data()], ignore_index=True)
        if i % (n_samples/10) == 0:
            print(f"calibrating gyro... {i}/{n_samples}")
    gyro_calib = df['gyro'].mean()
    df = pd.DataFrame()
    for i in range(n_samples):
        df = pd.concat([df, read_data()], ignore_index=True)
        if i % (n_samples/10) == 0:
            print(f"calibrating gyro... {i}/{n_samples}")
    
    df['accel'] = np.array([0, 0, 0])
    df['mag'] = np.array([0, 0, 0])
    return df

In [ ]:
n_samples = 20000
df = pd.DataFrame()
for i in range(n_samples):
    df = pd.concat([df, read_data()], ignore_index=True)
    if i % (n_samples/10) == 0:
        print(f"gathering data ... {i}/{n_samples}")
df

In [ ]:
df_mag = df['mag'].drop_duplicates().reset_index(drop=True)
df_mag

In [ ]:
(df_mag.max() - df_mag.min())/2

In [ ]:
# save to text with \t as delim
df_mag.to_csv('mag.txt', sep='\t', index=False, header=False)